In [72]:
import plotly.graph_objects as go

In [73]:
PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join('data')

In [74]:
import plotly.express as px
def get_color_scales():
    colors = px.colors.sequential.Greys.copy()
    # Remove extremes
    colors = colors[2:]
    colors.reverse()
    colors = colors[2:]
    background_color = colors[0]
    scale_color = colors[1:]
    return background_color, scale_color

In [75]:
# Setup global vars
FIG_WIDTH = 800
FIG_HEIGHT = 600
FIG_X_AXIS_MAX = 1000
FIG_Y_AXIS_MAX = 1000
FIG_SCALE = 1
BG_COLOR, COLOR_SCALE = get_color_scales()

In [162]:
# Set up the figure plot
def setup_figure(fig, set_background=True):
    #fig = go.Figure()
    fig.update_xaxes(range=[0, FIG_X_AXIS_MAX], showgrid=False)
    fig.update_yaxes(range=[0, FIG_Y_AXIS_MAX], showgrid=False)
    fig.update_layout(width=FIG_WIDTH, height=FIG_HEIGHT)
    if set_background:
        fig.layout.plot_bgcolor = 'rgb(37,37,37)'
        #fig.layout.paper_bgcolor = 'rgb(37,37,37)'
    return fig


In [113]:
# Function to add rectangle to the figure
def add_rectangle(fig, x0, y0, x1, y1, z, scale=1):
    x0 = x0 * scale
    x1 = x1 * scale
    y0 = y0 * scale
    y1 = y1 * scale
    fig = fig.add_shape(type='rect', x0=x0, x1=x1, y0=y0, y1=y1, fillcolor=get_color(z), line_color='rgb(0,0,0)', layer="below")
    label_text = 'z=' + str(z)
    fig = add_rect_text(fig, x0, y0, label_text)
    return fig

In [199]:
# Function to add z information to rectangles
def add_rect_text(fig, x0, y0, text, jitter=25):
    fig.add_trace(go.Scatter(
        x=[x0 + jitter],
        y=[y0 + jitter],
        text=[text],
        hoverinfo='skip',
        size=30,
        mode='text',
        
    ))
    return fig

In [104]:
import json
# Function to load rectange data from file
def json_to_list(file_name):
    file_path = os.path.join(DATA_PATH, file_name)
    data_list = []
    with open(file_path) as f:
        d = json.load(f)
        data = d['data']
        for rect in data:
            data_list.append(rect['rect'])
    return data_list

In [105]:
def get_color(z):
    return COLOR_SCALE[z-1]

In [118]:
def plot_rectangles(fig):
    data_list = json_to_list('rects.json')
    for data in data_list:
        x0, y0, x1, y1, z = data
        fig = add_rectangle(fig, x0=x0, y0=y0, x1=x1, y1=y1, z=z)
    return fig

In [170]:
import random
import pandas as pd

def generate_random_test_data(num=1000):
    random_x = [random.randint(0, FIG_X_AXIS_MAX) for _ in range(num)]
    random_y = [random.randint(0, FIG_Y_AXIS_MAX) for _ in range(num)]
    random_z = [random.randint(0, 4) for _ in range(num)]
    df = pd.DataFrame({'x': random_x, 'y': random_y, 'z': random_z})
    return df

In [217]:
def generate_player_data(num=100, jitter=30):
    df_list = []
    data_list = json_to_list('rects.json')
    for data in data_list:
        x0, y0, x1, y1, z = data
        random_x = [random.randint(x0-jitter, x1+jitter) for _ in range(num)]
        random_y = [random.randint(y0-jitter, y1+jitter) for _ in range(num)]
        random_z = []
        for x, y in zip(random_x, random_y):
            if x >= x0 and x <= x1 and y >= y0 and y <= y1:
                temp_z = z
            else:
                temp_z = 0
            random_z.append(temp_z)
        temp_df = pd.DataFrame({'x': random_x, 'y': random_y, 'z': random_z})
        df_list.append(temp_df)
    df = pd.concat(df_list)
    return df


In [218]:
df_success = generate_player_data()
df = pd.concat([df_success])
fig = px.scatter(df, x='x', y='y', color='z', color_continuous_scale=px.colors.sequential.Jet, opacity=0.3)
fig = setup_figure(fig)
fig = plot_rectangles(fig)
fig = fig.update_layout(showlegend=False)
fig.show()